In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
# from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

In [7]:
# tradingview lightweight-charts
from lightweight_charts import Chart
import asyncio
import nest_asyncio
# Apply nest_asyncio AT START to allow nested event loops (required for Jupyter)
nest_asyncio.apply()
#
from dotenv import load_dotenv

In [9]:
# load env vars
load_dotenv()

True

In [13]:
# #####---------------------#####s
# screen_date = '2025-02-11'
# screen_date = '2025-02-14'
screen_date = '2025-03-07'
# #####---------------------#####
# read pre market data from TradingView data via csv
trading_view_df = pd.read_csv(f"/Users/sudz4/Desktop/MATT-MATTHEW/matt-matthew/tv_screen_gap-up_{screen_date}.csv")

In [ ]:
# print(trading_view_df.columns)

In [14]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume',
    'Exponential Moving Average (3) 1 day', 'Exponential Moving Average (7) 1 day'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes',
    'Exponential Moving Average (3) 1 day',
    'Exponential Moving Average (7) 1 day'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")
display(smash_df.head(20))
# display(smash_df)
# # ALL SMASH
# display(smash_df)

19 stocks found.


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes,Exponential Moving Average (3) 1 day,Exponential Moving Average (7) 1 day
0,GENK,"GEN Restaurant Group, Inc.",Micro caps,35.166994,24.950884,2.242862e+08,6.838,6.3600,Restaurants,"NASDAQ Composite, NASDAQ Industrials",Consumer services,NASDAQ,2025-03-06,2025-05-20,4.422927e+06,181245.4,1.038812e+05,7.370697e+04,9.228891,0.149161,0.700024,66.005643,NaN,Buy,6.051778,5.767007
1,HE,"Hawaiian Electric Industries, Inc.",Mid caps,5.480769,2.884615,2.016123e+09,11.690,10.7000,Electric utilities,"Russell 2000, Russell 3000, Mini-Russell 2000",Utilities,NYSE,2025-02-21,2025-05-13,1.721400e+08,3277967.8,3.221347e+06,3.160280e+06,0.615135,0.350273,1.419038,4.253419,Neutral,Buy,11.085534,10.866144
2,W,Wayfair Inc.,Mid caps,5.015106,3.232628,4.489881e+09,35.380,34.1700,Specialty stores,"Nasdaq US Small Cap Growth, Russell 3000, Russell 1000",Retail trade,NYSE,2025-02-20,2025-05-01,9.255952e+07,4097554.8,3.888484e+06,4.322732e+06,0.226132,0.135305,0.497967,2.120350,Buy,Buy,34.753489,36.256646
3,HMY,Harmony Gold Mining Company Limited,Mid caps,4.884793,3.686636,6.807010e+09,11.740,11.2500,Precious metals,PHLX Gold/Silver Sector,Non-energy minerals,NYSE,2025-03-04,2025-08-28,6.217308e+08,5939878.6,7.029278e+06,5.927451e+06,0.307056,0.357636,1.542656,1.574088,Sell,Buy,11.249289,10.931582
4,BILI,Bilibili Inc.,Mid caps,3.321364,3.680431,9.402824e+09,23.925,23.1000,Packaged software,"NASDAQ Composite, NASDAQ Golden Dragon China, NASDAQ Computer",Technology services,NASDAQ,2025-02-20,2025-05-21,3.256927e+08,7202681.2,8.471389e+06,6.375355e+06,0.327352,0.134710,0.994837,1.095751,Buy,Buy,22.869211,21.979870
5,RIG,Transocean Ltd (Switzerland),Mid caps,3.249097,1.083032,2.636661e+09,3.000,2.8000,Contract drilling,"Russell 2000, Russell 3000, Mini-Russell 2000, PHLX Oil Service Sector",Industrial services,NYSE,2025-02-17,2025-05-05,7.329417e+08,36845176.2,3.167337e+07,2.516478e+07,0.222136,0.085626,1.072707,1.868460,Buy,Buy,2.883826,2.909175
6,VAL,Valaris Limited,Mid caps,3.158197,1.550388,2.627485e+09,36.990,35.3700,Contract drilling,"Russell 2000, S&P MidCap 400, Russell 3000, Mini-Russell 2000",Industrial services,NYSE,2025-02-19,2025-05-06,6.375611e+07,2986497.6,2.091118e+06,1.794485e+06,0.230744,0.070328,0.720904,1.828932,Buy,Buy,35.739870,35.933923
7,MNSO,MINISO Group Holding Limited,Mid caps,2.851218,3.110420,6.270070e+09,20.010,19.8900,Specialty stores,NASDAQ Golden Dragon China,Retail trade,NYSE,2024-11-29,2025-03-07,3.057084e+08,1084688.1,9.590060e+05,1.410100e+06,0.142019,0.032957,0.466699,0.713120,Strong buy,Strong buy,19.886808,20.130052
8,SSL,Sasol Ltd.,Mid caps,2.650602,3.855422,2.596925e+09,4.380,4.3100,Chemicals: major diversified,NaN,Process industries,NYSE,2025-02-24,2025-08-26,6.299792e+08,837923.3,1.081311e+06,1.103563e+06,0.242470,0.030311,0.416171,0.946714,Buy,Strong buy,4.288535,4.325294
9,TUYA,Tuya Inc.,Mid caps,2.325581,4.651163,2.399913e+09,3.785,3.6000,Packaged software,NASDAQ Golden Dragon China,Technology services,NYSE,2025-02-26,2025-05-08,4.514996e+08,5204418.4,5.954918e+06,2.587886e+06,0.391068,0.023242,0.544595,1.254313,Strong buy,Strong buy,3.607521,3.523259


In [15]:
# group and count by sector to new df called sector_count_df
sector_count_df = smash_df.groupby('Sector').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# print the date and time of the screen
print(f"Screening date: {screen_date}")
print(f"Total stocks returned: {len(smash_df)}")
display(sector_count_df)

Screening date: 2025-03-07
Total stocks returned: 19


,Sector,Count
4,Finance,3
9,Technology services,3
2,Consumer durables,2
5,Industrial services,2
8,Retail trade,2
0,Commercial services,1
1,Communications,1
3,Consumer services,1
6,Non-energy minerals,1
7,Process industries,1


In [16]:
smash_list = smash_df['Symbol'].tolist()
print(len(smash_list))
print(smash_list)

19
['GENK', 'HE', 'W', 'HMY', 'BILI', 'RIG', 'VAL', 'MNSO', 'SSL', 'TUYA', 'GT', 'WPP', 'BZ', 'TIGO', 'ZIM', 'OBDC', 'NIO', 'PLMR', 'RITM/PA']


In [ ]:
# Create a filter for EMA3 > EMA7
smash_ema_df = smash_df['Exponential Moving Average (3) 1 day'] > smash_df['Exponential Moving Average (7) 1 day']

# Apply the filter to smash_df
filtered_smash_df = smash_df[smash_ema_df]

# Print the number of stocks that meet the criteria
print(f"Number of stocks where EMA3 > EMA7: {len(filtered_smash_df)}")
print("\nFiltered stocks:")
print(filtered_smash_df['Symbol'].tolist())